In [2]:
setwd("/projects/CARDIPS/analysis/epigenome_resource")
source("analyses/jennifer/notebooks/functions.R")

set.seed(5366)
library(coloc)

This is coloc version 5.2.3



## **Prepare input for GWAS**

In [ ]:
qtls = fread("analyses/tim/gwas_coloc/scripts/qtls_090424.txt", data.table = T)
qtls$taskid = c(1:nrow(qtls))
qtls$path_exists = file.exists(qtls$path)

table(qtls$path_exists)

head(qtls,2)



In [10]:
table(qtls$path_exists)
qtls$egene = "TRUE"
qtls$analysis = qtls$data_type


 TRUE 
60306 

## **Launch job**

In [5]:
input_file="/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc/scripts/qtls_090424.txt"
manifest_file="/projects/CARDIPS/analysis/epigenome_resource/analyses/jennifer/gwas_independent/subset_manifest.txt"
script="path_to_05.1b.run_qtl_gwas_coloc.sh"
outdir="/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc2"
cmd = paste("qsub -N gwas_coloc2 -t", paste0("1-", nrow(qtls), ":1"), 
            "-tc 300 -pe smp 2 -V -cwd -o coloc_gwas2.out -e coloc_gwas2.err", script, input_file, manifest, outdir)

message(cmd)
# system(cmd)

ERROR: Error in nrow(qtls): object 'qtls' not found


## **concatenate all GWAS results**

In [14]:
tissues = c("iPSC", "PPC", "CVPC")
analyses = c("eqtls", "caqtls", "haqtls")

In [15]:
coloc_results_dir = "/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc2/"

In [17]:
table(qtls$tissue, qtls$data_type)

      
       caqtls eqtls haqtls
  CVPC  11239  4837   8937
  iPSC   9053  9012   1459
  PPC   10313  5456      0

In [4]:
manifest_file = "analyses/tim/gwas_coloc/scripts/manifest_subset.txt"
manifest = fread(manifest, data.table = F)


In [21]:
pipeline = "/projects/CARDIPS/analysis/epigenome_resource/analyses/jennifer/notebooks/08.04.summarize_gwas.R"
manifest_file = "analyses/tim/gwas_coloc/scripts/manifest_subset.txt"

for (t in tissues)
{
    for (a in analyses)
    {
        files = list.files(paste("analyses/tim/gwas_coloc2", a, t, sep = "/"))
        message(paste(a, t, length(files)), appendLF = F)
        if (length(files) > 0)
        {
            cmd = paste("Rscript", pipeline, 
                        "--analysis", a, 
                        "--tissue", t, 
                        "--manifest_file", manifest_file, 
                        "--coloc_results_dir", coloc_results_dir)
            
            log_out = paste(getwd(), "analyses/tim/gwas_coloc2/logs", paste(a, t, "summarize.out", sep = "_"), sep = "/")
            log_err = paste(getwd(), "analyses/tim/gwas_coloc2/logs", paste(a, t, "summarize.err", sep = "_"), sep = "/")
            run_qsub(name = paste(a, t, sep = "_"), cmd = cmd, threads = 4, log_out = log_out, log_err = log_err, exec = T)
        }
    }
}


eqtls iPSC 9006
echo "Rscript /projects/CARDIPS/analysis/epigenome_resource/analyses/jennifer/notebooks/08.04.summarize_gwas.R --analysis eqtls --tissue iPSC --manifest_file analyses/tim/gwas_coloc/scripts/manifest_subset.txt --coloc_results_dir /projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc2/" | qsub -N eqtls_iPSC -V -cwd -pe smp 4 -o /frazer01/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc2/logs/eqtls_iPSC_summarize.out -e /frazer01/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc2/logs/eqtls_iPSC_summarize.err

caqtls iPSC 8996
echo "Rscript /projects/CARDIPS/analysis/epigenome_resource/analyses/jennifer/notebooks/08.04.summarize_gwas.R --analysis caqtls --tissue iPSC --manifest_file analyses/tim/gwas_coloc/scripts/manifest_subset.txt --coloc_results_dir /projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc2/" | qsub -N caqtls_iPSC -V -cwd -pe smp 4 -o /frazer01/projects/CARDIPS/analysis/epigenome_resou

## **aggregate**

In [29]:
summary = as.data.frame(rbindlist(lapply(tissues, function(t)
{
    as.data.frame(rbindlist(lapply(analyses, function(a)
    {
        file = paste(coloc_results_dir, "results", paste(paste(a, t, "summary", sep = "_"), "txt", sep = "."), sep = "/")
        message(file)
        if (file.exists(file))
        {
            message(paste(t, a),  appendLF = F)
            fread(file, data.table = F)
        } else
        {
            message(paste("Missing", t, a))
        }
    })))
}))) %>% mutate(p.gwas = as.double(p.gwas), p.eqtl = as.double(p.eqtl)) %>% dplyr::rename(type = discovery_order) 

table(summary$analysis, summary$tissue)

/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc2//results/eqtls_iPSC_summary.txt

iPSC eqtls
/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc2//results/caqtls_iPSC_summary.txt

iPSC caqtls
/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc2//results/haqtls_iPSC_summary.txt

iPSC haqtls
/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc2//results/eqtls_PPC_summary.txt

PPC eqtls
/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc2//results/caqtls_PPC_summary.txt

PPC caqtls
/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc2//results/haqtls_PPC_summary.txt

Missing PPC haqtls

/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc2//results/eqtls_CVPC_summary.txt

CVPC eqtls
/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc2//results/caqtls_CVPC_summary.txt

CVPC caqtls
/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_col

        
           CVPC   iPSC    PPC
  caqtls 132222 106759 121376
  eqtls   58275 108835  65800
  haqtls 105604  17220      0